In [ ]:
import os
from datetime import datetime
# Essential Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score
import sqlite3

# Liste des fonctions

## f_requete_sql - Requete SQL

In [ ]:
def f_requete_sql (requete) :
    try:
        connexion = sqlite3.connect('./data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table", error)

## f_clusters_transform - Fonction conversion df en np.array

In [ ]:
def f_clusters_transform (df):
    """
    df : dataframe pandas
    """
    data = df.values.T
    names = df.columns
    return(data, names)

## f_clusters_coude - Fonction d'affichage de l'inertie en fonction du nombre de clusters

In [ ]:
def f_clusters_coude (df, k_max:int=15):
    """
    k_max : entier représentant le nombre max de clusters à tracer
    """
    data, names = f_clusters_transform(df)
    Sum_of_squared_distances = [] 
    K = range(1,30) 
    for k in K : 
        km = KMeans(n_clusters=k) 
        km = km.fit(data) 
        Sum_of_squared_distances.append(km.inertia_)

    plt.plot(K, Sum_of_squared_distances, 'bx-') 
    plt.xlabel('k') 
    plt.ylabel('Sum_of_squared_distances') 
    plt.title('Méthode du coude pour k optimal') 
    plt.show()

## f_clusters_fit - Fonction entrainement du modèle, retour des labels et profils clusters

In [ ]:
def f_clusters_fit (df, k:int = 4, n_init:int = 200, max_iter:int = 1000):
    """
    Entrainement et affichage de la distribution dans les clusters
    return :
    - cluster_table : df bss - cluster - r2
    - df_clusters_centers : df série temporelle des clusters
    """
    data, names = f_clusters_transform (df)
    km = KMeans(n_clusters=k, n_init=200, max_iter=1000)
    # Entraînement du modèle
    km.fit(data)
    # Définition des numéros de cluster
    labels = km.labels_+1
    # Profils des clusters
    cluster_centers = np.array(km.cluster_centers_)
    df_cluster_centers = pd.DataFrame(cluster_centers.T, index=df.index)
    for cluster in range(df_cluster_centers.shape[1]):
        df_cluster_centers.rename(columns = {cluster:f"cluster_{cluster+1}"}, inplace=True)
    
    # Création du dataframe de résultats
    cluster_table = pd.DataFrame()
    cluster_table["code_bss"] = names
    cluster_table.set_index("code_bss", inplace=True)
    cluster_table["cluster"] = labels
    for piezo in cluster_table.index :
        cluster_table.loc[piezo, "r2"] = r2_score(df.loc[:,piezo], cluster_centers[cluster_table.loc[piezo, "cluster"]-1])
    
    return cluster_table, df_cluster_centers

## f_plot_clusters - Fonction d'affichage des chroniques par cluster

In [ ]:
def f_plot_clusters (df, cluster_table, cluster_centers):
    
    data, names = f_clusters_transform (df)
    
    labels = cluster_table["cluster"]
    
    fig, axs = plt.subplots(len(set(labels)),1,figsize=(20,25))
    row_i=0

    for label in set(labels):
        for i in range(len(labels)):
            if(labels[i]==label):
                axs[row_i].plot(df.iloc[:,i],alpha=0.4)

        axs[row_i].plot(cluster_centers.iloc[:,label-1],c="red")
        axs[row_i].set_title("Cluster "+str(row_i+1))
        row_i+=1
    plt.show()

## f_plot_all - Fonction d'affichage individuel des chroniques et leur cluster associé

In [ ]:
def f_plot_all (df, cluster_table, df_cluster_centers) :

    data, names = f_clusters_transform (df)

    if df.shape[1]%4 == 0:
        nbre_lignes = df.shape[1]//4
    else : 
        nbre_lignes = df.shape[1]//4 + 1

    fig, axs = plt.subplots(nbre_lignes,4,figsize=(25,200))
    #fig.suptitle('Series')
    for i in range(nbre_lignes):
        for j in range(4):
            if i*4+j+1>df.shape[1]: # pass the others that we can't fill
                continue
            axs[i, j].plot(df.iloc[:,i*4+j])
            axs[i, j].plot(df_cluster_centers.iloc[:,cluster_table.iloc[i*4+j,0]-1])
            
            axs[i, j].set_title(df.columns[i*4+j] + f" - r² = {round(cluster_table.loc[df.columns[i*4+j],'r2'],2)}")
            
    plt.show()

## f_r2min - Fonction de calcul du seuil minimal r2

In [ ]:
def f_r2min (cluster_table):
    # Calcul de r2min

    Q1 = cluster_table["r2"].quantile(0.25, "higher")
    Q3 = cluster_table["r2"].quantile(0.75, "lower")

    # Ecart inter-quartile
    IQM = (Q3-Q1)

    # Seuil minimal de conservation des chroniques (Q1-1.5*IQM)

    r2min = Q1-1.5*IQM
    return r2min

# Exploitation

## Chargement des données

In [ ]:
region = "ARA"

# Import du csv et conversion en dataframe
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
data = pd.read_csv(f"./data/piezo_clustering/{region}.csv", sep = ";", parse_dates=True, index_col="date_mesure", date_parser=custom_date_parser)

## Analyse du nombre optimal de clusters

In [ ]:
f_clusters_coude(data)

In [ ]:
nbre_clusters = 6

## Création des clusters

In [ ]:
cluster_table, cluster_centers = f_clusters_fit(data, nbre_clusters, 1000, 8000)

In [ ]:
sns.countplot(cluster_table.cluster)

## Affichage des clusters

In [ ]:
f_plot_clusters(data, cluster_table, cluster_centers)

In [ ]:
f_plot_all(data, cluster_table, cluster_centers)

## Analyse r2

In [ ]:
plt.plot(cluster_table["r2"].sort_values())

## Filtre des chroniques avec score insuffisant

In [ ]:
# nbre de chroniques sous le seuil
r2min = f_r2min(cluster_table)
nbre_elim = cluster_table.loc[cluster_table.r2 < r2min, 'r2'].count()
iteration = 1

while nbre_elim != 0 :
    # chroniques sous le seuil passées à cluster 0
    r2min = f_r2min(cluster_table)
    cluster_table.loc[cluster_table.r2 < r2min, "cluster"] = 0
    
    # Selection des clusters non zéro
    data = data.loc[:,cluster_table.loc[cluster_table.cluster != 0].index]
    
    # Nouveau clustering
    cluster_table, cluster_centers = f_clusters_fit(data, nbre_clusters, 1000, 8000)
    
    nbre_elim = cluster_table.loc[cluster_table.r2 < r2min, 'r2'].count()
    
    print(f"Itération {iteration} : {nbre_elim} chronique(s) éliminée(s).")
    iteration += 1
    

In [ ]:
plt.plot(cluster_table["r2"].sort_values())

In [ ]:
f_plot_clusters(data, cluster_table, cluster_centers)

In [ ]:
cluster_table.sort_values(by='r2').head()

# Ajout à la bdd

In [ ]:
# On renomme les noms de cluster par rapport à la région
for index, column in enumerate (cluster_centers.columns):
    cluster_centers.rename(columns={column : f"{region}_{index+1}"}, inplace=True)

In [ ]:
data.to_csv(f"./data/piezo_clustering/{region}.csv", sep = ";")

In [ ]:
cluster_centers.to_csv(f"./data/piezo_clustering/{region}_clusters.csv", sep = ";")

In [ ]:
requete = """
                    ALTER TABLE data_piezo
                    ADD cluster_sklearn_kmeans_5_v1 VARCHAR(6) DEFAULT NULL
                    """
f_requete_sql(requete)

In [ ]:
# Création de listes de nuéro de cluster avec le nom des piezos
# Ajout dans la bdd
for label in set(cluster_table.cluster):
    exec(f'cluster = tuple(cluster_table.loc[cluster_table.cluster == {label}, "cluster"].index.to_list())')
    requete = f"""
        UPDATE data_piezo
        SET cluster_sklearn_kmeans_5_v1 = '{region + "_" + str(label)}'
        WHERE code_bss IN {cluster}
        """
    f_requete_sql(requete)

## Suppresion colonne

requete = """
                    ALTER TABLE data_piezo
                    DROP cluster_sklearn_kmeans_5_v1
                    """
f_requete_sql(requete)